## Add DB connection test

In [8]:
from sqlalchemy import create_engine, text


# ========= 1. EDIT THESE VALUES FOR YOUR MACHINE =========

DB_USER = "vivekarya"          # change if your username is different
DB_PASSWORD = "Ap28bf9456" # 🔴 CHANGE THIS
DB_HOST = "localhost"
DB_PORT = 5432
DB_NAME = "postgres"     # make sure this database exists

# ========= 2. HELPER: MAKE DB CONNECTION =========

def get_engine():
    url = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
    engine = create_engine(url)
    return engine

# ========= 3. MAIN: TEST DB CONNECTION ONLY =========

def main():
    engine = get_engine()
    try:
        with engine.connect() as conn:
            result = conn.execute(text("SELECT 1;")).fetchall()
            print("✅ DB connection OK:", result)
    except Exception as e:
        print("❌ DB connection FAILED")
        print(e)

if __name__ == "__main__":
    main()


✅ DB connection OK: [(1,)]


In [18]:
import os
from datetime import datetime, timedelta

import pandas as pd
from nemosis import dynamic_data_compiler

# ---------- SETTINGS (EDIT DATES IF YOU WANT) ----------
table_name = "DISPATCHREGIONSUM"
raw_data_cache_dir = "data_raw"

# test for 1–2 months only to keep it light
start_date = datetime(2025, 1, 1, 0, 0, 0)
end_date   = datetime(2025, 3, 1, 0, 0, 0)   # not included (so Jan + Feb)

os.makedirs(raw_data_cache_dir, exist_ok=True)

current = start_date

print("\n--- NEMOSIS DATA CHECK (ALL REGIONS, NO DB) ---")

while current < end_date:
    # month boundaries
    next_month = (current.replace(day=28) + timedelta(days=4)).replace(day=1)
    end_of_month = next_month - timedelta(seconds=1)

    start_str = current.strftime("%Y/%m/%d %H:%M:%S")
    end_str   = end_of_month.strftime("%Y/%m/%d %H:%M:%S")

    print(f"\n📅 Fetching {start_str} → {end_str} for {table_name} ...")

    try:
        df = dynamic_data_compiler(
            start_str,
            end_str,
            table_name,
            raw_data_cache_dir,   # raw_data_cache
        )

        print("   ✅ Rows fetched:", len(df))

        if len(df) > 0:
            # show region IDs available
            if "REGIONID" in df.columns:
                print("   🌏 Regions in this chunk:", df["REGIONID"].unique())
            else:
                print("   ⚠️ 'REGIONID' column not found. Columns are:")
                print("      ", df.columns.tolist())

            # show a tiny sample
            print("   📊 Sample rows:")
            print(df.head(3))

        del df

    except Exception as e:
        print(f"   ❌ Error for {current.strftime('%B %Y')}: {e}")

    current = next_month

print("\n--- NEMOSIS CHECK COMPLETE ---")



--- NEMOSIS DATA CHECK (ALL REGIONS, NO DB) ---

📅 Fetching 2025/01/01 00:00:00 → 2025/01/31 23:59:59 for DISPATCHREGIONSUM ...
INFO: Compiling data for table DISPATCHREGIONSUM
INFO: Downloading data for table DISPATCHREGIONSUM, year 2024, month 12
INFO: Creating feather file for DISPATCHREGIONSUM, 2024, 12
INFO: Downloading data for table DISPATCHREGIONSUM, year 2025, month 01
INFO: Creating feather file for DISPATCHREGIONSUM, 2025, 01
INFO: Returning DISPATCHREGIONSUM.
   ✅ Rows fetched: 44635
   🌏 Regions in this chunk: ['NSW1' 'QLD1' 'SA1' 'TAS1' 'VIC1']
   📊 Sample rows:
       SETTLEMENTDATE REGIONID  DISPATCHINTERVAL  INTERVENTION  TOTALDEMAND  \
0 2025-01-01 00:05:00     NSW1       20241231241             0      7251.07   
1 2025-01-01 00:05:00     QLD1       20241231241             0      6444.99   
2 2025-01-01 00:05:00      SA1       20241231241             0      1379.47   

   AVAILABLEGENERATION  AVAILABLELOAD  DEMANDFORECAST  DISPATCHABLEGENERATION  \
0          12034.7

### Final Ingestion  2025 data 

In [25]:
import os
from datetime import datetime, timedelta

import pandas as pd
from sqlalchemy import create_engine
from nemosis import dynamic_data_compiler

# ---------- 1. POSTGRES SETTINGS ----------
DB_USER = "vivekarya"          # change if your username is different
DB_PASSWORD = "Ap28bf9456" # 🔴 CHANGE THIS
DB_HOST = "localhost"
DB_PORT = 5432
DB_NAME = "postgres"  

def get_engine():
    url = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
    return create_engine(url)

# ---------- 2. NEMOSIS SETTINGS ----------
table_name = "DISPATCHREGIONSUM"
raw_data_cache_dir = "data_raw"
os.makedirs(raw_data_cache_dir, exist_ok=True)

# Use a small test range first (you can change later)
start_date = datetime(2025, 1, 1, 0, 0, 0)
end_date = datetime(2025, 11, 10, 0, 0, 0)  # Jan + Feb 2025

def main():
    engine = get_engine()

    # Test DB connection
    with engine.connect() as conn:
        result_1=conn.execute(text("SELECT 1;")).fetchall()
        print("✅ DB connection OK:", result_1)

    # (Optional) Clear old table so you don't duplicate rows during testing
    with engine.connect() as conn:
        conn.execute(text("DROP TABLE IF EXISTS dispatch_region_5min;"))
     ##execute(text("SELECT 1;")).fetchall()
        print("🧹 Dropped old table dispatch_region_5min (if existed).")

    current = start_date

    print("\n--- Starting NEMOSIS → Postgres Ingestion (ALL REGIONS) ---")

    total_inserted = 0

    while current < end_date:
        # Month boundaries
        next_month = (current.replace(day=28) + timedelta(days=4)).replace(day=1)
        end_of_month = next_month - timedelta(seconds=1)

        start_str = current.strftime("%Y/%m/%d %H:%M:%S")
        end_str   = end_of_month.strftime("%Y/%m/%d %H:%M:%S")

        print(f"\n📅 Fetching {start_str} → {end_str} for {table_name} (ALL REGIONS)...")

        try:
            # 1️⃣ Fetch from NEMOSIS
            df = dynamic_data_compiler(
                start_str,
                end_str,
                table_name,
                raw_data_cache_dir,   # raw_data_cache
            )

            print(f"   ✅ Rows fetched this month: {len(df)}")

            if len(df) == 0:
                print("   ℹ️ No rows for this month, skipping.")
                current = next_month
                continue

            # 2️⃣ Make sure REGIONID exists
            if "REGIONID" not in df.columns:
                print("   ⚠️ 'REGIONID' column missing, available columns:")
                print("      ", df.columns.tolist())
                current = next_month
                continue

            # 3️⃣ Keep only columns we need
            df["SETTLEMENTDATE"] = pd.to_datetime(df["SETTLEMENTDATE"])
            df_clean = df[["SETTLEMENTDATE", "REGIONID", "TOTALDEMAND"]].copy()

            # 4️⃣ Rename columns for DB
            df_clean = df_clean.rename(columns={
                "SETTLEMENTDATE": "settlement_ts",
                "REGIONID": "region_id",
                "TOTALDEMAND": "total_demand",
                
            })

            # 5️⃣ Insert into Postgres
            df_clean.to_sql(
                "dispatch_region_5min",
                engine,
                if_exists="append",
                index=False,
            )

            print(f"   ✅ Inserted {len(df_clean)} rows for month {current.strftime('%B %Y')}.")
            total_inserted += len(df_clean)

            # free memory
            del df
            del df_clean

        except Exception as e:
            print(f"   ❌ Error while processing {current.strftime('%B %Y')}: {e}")

        current = next_month

    print("\n--- Ingestion complete! ---")
    print(f"📦 Total rows inserted into dispatch_region_5min: {total_inserted}")

if __name__ == "__main__":
    main()


✅ DB connection OK: [(1,)]
🧹 Dropped old table dispatch_region_5min (if existed).

--- Starting NEMOSIS → Postgres Ingestion (ALL REGIONS) ---

📅 Fetching 2025/01/01 00:00:00 → 2025/01/31 23:59:59 for DISPATCHREGIONSUM (ALL REGIONS)...
INFO: Compiling data for table DISPATCHREGIONSUM
INFO: Returning DISPATCHREGIONSUM.
   ✅ Rows fetched this month: 44635
   ✅ Inserted 44635 rows for month January 2025.

📅 Fetching 2025/02/01 00:00:00 → 2025/02/28 23:59:59 for DISPATCHREGIONSUM (ALL REGIONS)...
INFO: Compiling data for table DISPATCHREGIONSUM
INFO: Returning DISPATCHREGIONSUM.
   ✅ Rows fetched this month: 40315
   ✅ Inserted 40315 rows for month February 2025.

📅 Fetching 2025/03/01 00:00:00 → 2025/03/31 23:59:59 for DISPATCHREGIONSUM (ALL REGIONS)...
INFO: Compiling data for table DISPATCHREGIONSUM
INFO: Returning DISPATCHREGIONSUM.
   ✅ Rows fetched this month: 44635
   ✅ Inserted 44635 rows for month March 2025.

📅 Fetching 2025/04/01 00:00:00 → 2025/04/30 23:59:59 for DISPATCHREGION